# Convert to Python

In [1]:
import os
import json
from nbconvert import PythonExporter
import nbformat

# Directories using relative paths
current_directory = os.getcwd()
src_directory = os.path.join(current_directory, '..', 'Package', 'notebooks')
dest_directory = os.path.join(current_directory, '..', 'Package', 'my_package')

# Function to check if a file is valid JSON
def is_valid_json(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            json.load(file)
        return True
    except (ValueError, json.JSONDecodeError):
        return False

# Function to convert .ipynb to .py and clean up the output
def convert_notebook_to_script(notebook_path, script_path):
    try:
        with open(notebook_path, 'r', encoding='utf-8') as notebook_file:
            notebook_content = nbformat.read(notebook_file, as_version=4)
        python_exporter = PythonExporter()
        script, _ = python_exporter.from_notebook_node(notebook_content)

        # Clean up the script
        lines = script.split('\n')
        cleaned_lines = [line for line in lines if not line.startswith('#!') and not line.startswith('# In')]

        with open(script_path, 'w', encoding='utf-8') as script_file:
            script_file.write('\n'.join(cleaned_lines))
    except Exception as e:
        print(f"Failed to convert {notebook_path}: {e}")

# Walk through the source directory
for root, dirs, files in os.walk(src_directory):
    for file in files:
        if file.endswith(".ipynb"):
            notebook_path = os.path.join(root, file)
            if os.path.getsize(notebook_path) > 0 and is_valid_json(notebook_path):
                relative_path = os.path.relpath(root, src_directory)
                script_dest_directory = os.path.join(dest_directory, relative_path)
                script_file_name = file.replace(".ipynb", ".py")
                script_path = os.path.join(script_dest_directory, script_file_name)

                # Create the destination directory if it doesn't exist
                os.makedirs(script_dest_directory, exist_ok=True)

                # Convert the notebook and write to the destination
                convert_notebook_to_script(notebook_path, script_path)
                # print(f"Converted {notebook_path} to {script_path}")
            else:
                print(f"Skipped {notebook_path}: not a valid JSON file or empty")

print("Finished converting files from .ipynb to .py")


Finished converting files from .ipynb to .py


# Create Root Main File

In [2]:
import os

# Define the directory paths
current_directory = os.getcwd()
package_dir = os.path.join(current_directory, '..', 'Package', 'my_package')
main_file_path = os.path.join(package_dir, "main.py")  # Place main.py in the my_package directory

# Function to create the content for the main.py file
def create_main_file():
    # Get the list of folders inside the package_dir, excluding __pycache__
    folders = [f for f in os.listdir(package_dir) if os.path.isdir(os.path.join(package_dir, f)) and f != '__pycache__']
    
    # Start writing the main.py content
    with open(main_file_path, 'w', encoding='utf-8') as main_file:
        for folder in folders:
            # Replace spaces in folder names for imports
            import_statement = f"from .{folder.replace(' ', '_').replace('-', '_')} import *\n"
            main_file.write(import_statement)

# Run the function to create the main.py file
if __name__ == "__main__":
    create_main_file()

print('Finished creating root main.py file')

Finished creating root main.py file


# Zip Package

In [3]:
import os
import shutil
import sys

# Suppress output
original_stdout = sys.stdout
sys.stdout = open(os.devnull, 'w')

# Define the directory to be zipped and the output ZIP file path
current_directory = os.getcwd()
source_dir = os.path.join(current_directory, '..', 'Package', 'my_package')
zip_file_path = os.path.join(current_directory, '..', 'Package', 'my_package.zip')

# Create a ZIP file from the source directory
shutil.make_archive(zip_file_path[:-4], 'zip', source_dir)

# Restore standard output
sys.stdout.close()
sys.stdout = original_stdout
